# 控制组-受到真实新闻的影响

## 读取数据并处理

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk
import re

In [18]:
df_control_row = pd.read_csv('df_control_clear.csv',dtype={'股票代码': str})


检视重复出现的股票代码

In [19]:
# 统计股票代码的出现次数
code_counts = df_control_row["股票代码"].value_counts()

# 找出重复的股票代码及其出现次数
duplicate_code_counts = code_counts[code_counts > 1]
print("重复的股票代码及其出现次数：")
print(duplicate_code_counts)


重复的股票代码及其出现次数：
股票代码
002310    9
002717    6
601186    5
300197    5
300021    4
601238    3
600340    3
603636    3
300335    3
000018    3
300376    3
300262    3
600567    3
600068    3
300601    3
603658    3
300072    3
300145    3
300116    3
300362    3
000597    2
600530    2
300202    2
002205    2
300596    2
300198    2
300007    2
002153    2
000413    2
300223    2
300281    2
300356    2
300133    2
002431    2
300075    2
000869    2
002616    2
300370    2
002665    2
600835    2
300001    2
300397    2
300037    2
300210    2
002239    2
300094    2
601608    2
300363    2
300265    2
300225    2
002037    2
000558    2
603283    2
601179    2
300676    2
300334    2
002628    2
600737    2
300220    2
300708    2
Name: count, dtype: int64


一个复杂的工作

In [ ]:
import os
import pandas as pd

# Step 1: 从文件 A 中提取重复的主码
def extract_duplicate_primary_keys(file_a_path):
    df = pd.read_excel(file_a_path,sheet_name=0,dtype={'股票代码': str})
    # 检查文件是否包含必要列
    if "股票代码" not in df.columns or "新闻发布时间" not in df.columns:
        raise ValueError("文件 A 必须包含 '股票代码' 和 '日期' 列")

    # 找到重复的股票代码
    duplicate_codes = df["股票代码"].value_counts()[df["股票代码"].value_counts() > 1].index
    duplicates = df[df["股票代码"].isin(duplicate_codes)]

    # 创建主码列
    duplicates["主码"] = duplicates["股票代码"] + "_" + duplicates["新闻发布时间"].dt.date.astype(str)
    return duplicates[["股票代码", "新闻发布时间", "主码"]]

# Step 2: 从文件夹中提取主码
def extract_primary_keys_from_folder(folder_path):
    files = os.listdir(folder_path)
    folder_primary_keys = []
    for file in files:
        if file.endswith(".csv") and "_" in file:
            try:
                # 提取文件名中的股票代码和日期
                stock_code, date = file.split("_")
                date = date.split(".")[0]  # 去掉扩展名
                folder_primary_keys.append(f"{stock_code}_{date}")
            except ValueError:
                print(f"文件名格式不符合预期: {file}")
    return set(folder_primary_keys)

# Step 3: 比对主码，找出不存在的
def find_missing_primary_keys(folder_primary_keys, duplicate_primary_keys):
    missing = duplicate_primary_keys[~duplicate_primary_keys["主码"].isin(folder_primary_keys)]
    return missing[["股票代码", "新闻发布时间"]]  # 只保留股票代码和日期

# Step 4: 主程序
def main():
    file_a_path = "data_china.xlsx"  # 替换为文件 A 的路径
    folder_path = "D:\mycodelife\workshop\\fake_finance\\ready_crawler\dull"  # 替换为文件夹路径

    # Step 1: 提取文件 A 中的重复主码
    duplicate_primary_keys = extract_duplicate_primary_keys(file_a_path)
    print(duplicate_primary_keys)
    # Step 2: 提取文件夹中的主码
    folder_primary_keys = extract_primary_keys_from_folder(folder_path)
    print(folder_primary_keys)

    # Step 3: 找出文件夹中不存在的主码
    missing_primary_keys = find_missing_primary_keys(folder_primary_keys, duplicate_primary_keys)

    # Step 4: 保存结果到 Excel 文件
    missing_primary_keys.to_excel("missing_primary_keys.xlsx", index=False)
    print("不存在的主码已保存到 missing_primary_keys.xlsx")

# 执行主程序
if __name__ == "__main__":
    main()


C:\Users\86198\AppData\Local\Temp\ipykernel_20148\816067059.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicates["主码"] = duplicates["股票代码"] + "_" + duplicates["新闻发布时间"].dt.date.astype(str)


       股票代码     新闻发布时间                 主码
5    600569 2015-01-23  600569_2015-01-23
6    000662 2015-01-26  000662_2015-01-26
9    600487 2015-02-01  600487_2015-02-01
12   600060 2015-03-02  600060_2015-03-02
15   600265 2015-03-24  600265_2015-03-24
..      ...        ...                ...
473  600418 2021-07-27  600418_2021-07-27
483  601012 2022-03-01  601012_2022-03-01
485  601012 2022-04-21  601012_2022-04-21
487  603023 2022-06-24  603023_2022-06-24
494  000980 2022-11-04  000980_2022-11-04

[176 rows x 3 columns]
{'300919_2022-11-04', '688567_2020-09-22', '600132_2019-01-02', '000413_2019-01-25', '600530_2016-04-25', '002411_2020-10-29', '002224_2020-06-05', '600416_2022-02-22', '600057_2015-01-09', '002359_2018-12-11', '600276_2020-09-22', '300596_2021-06-21', '600118_2019-09-06', '688661_2022-09-01', '000413_2019-04-09', '600609_2019-05-29', '002557_2020-04-08', '603555_2020-07-17', '000541_2020-04-26', '300756_2020-09-15', '002082_2019-06-25', '002226_2020-05-22', '600086_2

PermissionError: [Errno 13] Permission denied: 'missing_primary_keys.xlsx'

按照真实性删掉虚假新闻后所得

In [8]:
filtered_df = df_control_row[df_control_row["真实性（虚假1真实0）"] == 0]

# 选择保留的列
columns_to_drop = ["公告地址", "公司简称", "原文链接","真实性（虚假1真实0）"]
filtered_df = filtered_df.drop(columns=columns_to_drop)
# 将筛选后的数据保存到新的 Excel 文件
filtered_df.to_csv("df_control_clear.csv", index=False,encoding="utf-8-sig")

## 新闻文本情绪测度

### 数据预处理
先读取原文和标题

In [16]:
df_control_clear = pd.read_csv("df_control_clear.csv",dtype={'股票代码':str})

In [19]:
df_content = df_control_clear[["股票代码","原文","新闻发布时间"]]

In [17]:
df_title = df_control_clear[["股票代码","标题","新闻发布时间"]]

切割文本长度

In [20]:
def split_text(row, max_length=500):
    """
    将原文按 max_length 分割，并为每段保留对应的主码（包括股票代码和新闻发布时间）。
    """
    text = row["原文"]
    code = row["股票代码"]
    date = row["新闻发布时间"]
    
    # 按长度 max_length 分割
    chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]
    
    # 返回主码（股票代码、新闻发布时间）和分割后的内容
    return [{"股票代码": code, "新闻发布时间": date, "分段内容": chunk} for chunk in chunks]

# 应用函数，将结果展开为一个新 DataFrame
result = df_content.apply(split_text, axis=1)  # 每行应用
df_content_cut = pd.DataFrame([item for sublist in result for item in sublist])



### 调用BERT
同虚假新闻

In [21]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline

tokenizer = AutoTokenizer.from_pretrained("bardsai/finance-sentiment-zh-fast")
model = AutoModelForSequenceClassification.from_pretrained("bardsai/finance-sentiment-zh-fast")
nlp_bardsai = pipeline("text-classification", model="bardsai/finance-sentiment-zh-fast")

c:\ProgramData\Anaconda3\envs\dirty_test\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


对虚假新闻标题进行情感分析

In [31]:
# 对每一段进行情绪分析
df_title["情绪分析结果"] = df_title["标题"].apply(nlp_bardsai)

# 分解情绪分析结果并扩展列
df_title["情绪标签"] = df_title["情绪分析结果"].apply(lambda x: x[0]["label"])
df_title["情绪分数"] = df_title["情绪分析结果"].apply(lambda x: x[0]["score"])
df_title = df_title.drop(columns=["情绪分析结果"])

# 保存为 CSV 文件
df_title.to_csv("badrdsai_result_title_control_group.csv", index=False, encoding="utf-8-sig")

print("CSV 文件已生成：情绪分析结果.csv")

CSV 文件已生成：情绪分析结果.csv


对新闻文本进行情感分析

In [23]:
# 对每一段进行情绪分析
df_content_cut["情绪分析结果"] = df_content_cut["分段内容"].apply(nlp_bardsai)

# 分解情绪分析结果并扩展列
df_content_cut["情绪标签"] = df_content_cut["情绪分析结果"].apply(lambda x: x[0]["label"])
df_content_cut["情绪分数"] = df_content_cut["情绪分析结果"].apply(lambda x: x[0]["score"])
df_content_cut = df_content_cut.drop(columns=["情绪分析结果"])

# 保存为 CSV 文件
df_content_cut.to_csv("badrdsai_result_content_control_group.csv", index=False, encoding="utf-8-sig")

print("CSV 文件已生成：情绪分析结果.csv")

CSV 文件已生成：情绪分析结果.csv


## 处理情绪分析结果

In [28]:
df_content_analysis = pd.read_csv("badrdsai_result_content_control_group.csv",dtype={"股票代码":str})
df_content_analysis

,股票代码,新闻发布时间,分段内容,情绪标签,情绪分数
0,000156,2015-01-04,华数传媒(000156)周日晚间公告称，其全资子公司浙江华数传媒资本管理有限公司与金石投资有...,neutral,0.999913
1,002037,2015-01-04,久联发展1月4日晚间公告称，公司于2014年12月30日收到公司下属全资子公司新联爆破集团关...,neutral,0.999914
2,002055,2015-01-07,得润电子7日公告，公司拟在广东省鹤山市建设运营得润工业园项目，以进一步扩大公司产能和规模，完...,positive,0.999719
3,300220,2015-01-08,金运激光1月8日晚间公告称，公司控股子公司武汉落地创意文化传播有限公司（简称“落地创意”）1...,neutral,0.999891
4,600057,2015-01-09,象屿股份（600057）9日晚间公告，近日，公司控股股东象屿集团与黑龙江省粮食局签署关于粮食...,neutral,0.999913
...,...,...,...,...,...
718,002752,2022-09-13,\n涉嫌证券市场操纵 昇兴股份董事长被罚100万\n2022-09-13 02:25 ...,neutral,0.999909
719,002752,2022-09-13,金属容器委员会第七届委员会副主任委员、福州市外商投资企业协会第六届理事会副会长。现任昇兴股份...,neutral,0.999903
720,002752,2022-09-13,赔长期处于探索阶段。据不完全统计，目前，国内仅有两例投资者以被告操纵证券市场为由索赔胜诉。\...,neutral,0.999904
721,002752,2022-09-13,东、实际控制人李广胜家属的通知，李广胜因涉嫌操纵证券、期货市场罪被湖州市公安局采取刑事拘留措...,neutral,0.999913


In [30]:
# 新增列：判断是否为 positive、neutral 和 negative
df_content_analysis["neutral"] = df_content_analysis["情绪标签"].apply(lambda x: 1 if x == "neutral" else 0)
df_content_analysis["positive"] = df_content_analysis["情绪标签"].apply(lambda x: 1 if x == "positive" else 0)
df_content_analysis["negative"] = df_content_analysis["情绪标签"].apply(lambda x: 1 if x == "negative" else 0)

# 分组统计
result = df_content_analysis.groupby(["股票代码","新闻发布时间"]).agg(
    # 中性均值，若无中性情绪，填充为 0
    中性均值=("情绪分数", lambda x: x[df_content_analysis.loc[x.index, "neutral"] == 1].mean() if any(df_content_analysis.loc[x.index, "neutral"] == 1) else 0),
    # 积极均值，若无积极情绪，填充为 0
    积极均值=("情绪分数", lambda x: x[df_content_analysis.loc[x.index, "positive"] == 1].mean() if any(df_content_analysis.loc[x.index, "positive"] == 1) else 0),
    # 消极均值，若无消极情绪，填充为 0
    消极均值=("情绪分数", lambda x: x[df_content_analysis.loc[x.index, "negative"] == 1].mean() if any(df_content_analysis.loc[x.index, "negative"] == 1) else 0),
    # 中性占比
    中性占比=("neutral", "mean"),
    # 积极占比
    积极占比=("positive", "mean"),
    # 消极占比
    消极占比=("negative", "mean")
).reset_index()

# 计算每类加权分数
result["中性分数"] = result["中性均值"] * result["中性占比"]
result["积极分数"] = result["积极均值"] * result["积极占比"]
result["消极分数"] = result["消极均值"] * result["消极占比"]

# 仅保留所需列
result = result[["股票代码","新闻发布时间","中性分数", "积极分数", "消极分数"]]

# 保存结果到 CSV 文件
result.to_csv("bardsai_analysis_content_control.csv", index=False, encoding="utf-8-sig")
